<a href="https://colab.research.google.com/github/mogranji/datasets/blob/master/Another_copy_of_IE7615_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Import all the required Libraries

In [ ]:
!pip install Augmentor

In [ ]:
!pip install scikit-plot

In [ ]:
# Import Tensorflow and Keras
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D,BatchNormalization
from tensorflow.keras.utils import image_dataset_from_directory

#Image processing libraries
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.models import load_model
import pickle
from PIL import Image
import cv2
import Augmentor

#import libraries for data processing and classification evaluate\
import pandas as pd
import numpy as np
from sklearn import metrics
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

#Visualization tools
import matplotlib.pyplot as plt
import seaborn as sns
import scikitplot as skplt

#import system library
import pandas as pd
import os

from sklearn.preprocessing import LabelEncoder

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Label images

In [ ]:
import os
import pandas as pd

# Define the directory where your images are located
image_dir = '/content/drive/MyDrive/IE7615/Cropped Classified'  # Change this to your image directory path

def ImageLabel(image_dir):

  # Initialize empty lists to store image paths and labels
  image_paths = []
  labels = []

  # Loop through the subdirectories (class labels)
  for label in os.listdir(image_dir):
      label_path = os.path.join(image_dir, label)
      if os.path.isdir(label_path):
          for image in os.listdir(label_path):
              # Ensure the file is an image (you can add more extensions as needed)
              if image.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp')):
                  image_path = os.path.join(label_path, image)
                  image_paths.append(image_path)
                  labels.append(label)
  return(image_paths, labels)

In [ ]:
imagePath,labels =ImageLabel(image_dir)
# Create a DataFrame from the lists
df = pd.DataFrame({'imagePath': imagePath, 'label': labels})
#Show the df
df

In [ ]:
df['label'].unique()

In [ ]:
def subStrRemoval():
  subStr = ['01 ', '02 ','03 ', '04 ','05 ']
  for i in subStr:
    df['label'] = df['label'].str.replace(i, '')
  return df

In [ ]:
df1 = subStrRemoval()
df1

# Preprocessing

In [ ]:
encoder = LabelEncoder()
encoder.fit(df1['label'])
encoded_y = encoder.transform(df1['label'])
counts = np.bincount(encoded_y)
print(counts)
fig, ax = plt.subplots()
plt.bar(list(range(5)), counts)
ax.set_xticklabels(('', 'Basophil', 'Eosinophil', 'Lymphocyte', 'Monocyte', 'Neutrophil'))
ax.set_ylabel('Counts')

# Split and Augment Images

## Image Balancing

In [ ]:
from imblearn.over_sampling import RandomOverSampler

In [ ]:
def balance_set(df, x_cols, y_cols):
    ros = RandomOverSampler(random_state=42)

    x_multi, y_multi = ros.fit_resample(df[x_cols], df[y_cols].values)
    data = pd.concat([x_multi, pd.DataFrame(y_multi, columns= y_cols)], axis=1)
    return data


In [ ]:
train_multi = balance_set(df,
                          x_cols = ["imagePath"],
                          y_cols = ['label'])

train_multi

In [ ]:
encoder = LabelEncoder()
encoder.fit(train_multi['label'])
encoded_y = encoder.transform(train_multi['label'])
counts = np.bincount(encoded_y)
print(counts)
fig, ax = plt.subplots()
plt.bar(list(range(5)), counts)
ax.set_xticklabels(('', 'Basophil', 'Eosinophil', 'Lymphocyte', 'Monocyte', 'Neutrophil'))
ax.set_ylabel('Counts')

In [ ]:
Monocyte = train_multi[train_multi['label'] == 'Monocyte'].count()[1]
Eosinophil = train_multi[train_multi['label'] == 'Eosinophil'].count()[1]
Basophil = train_multi[train_multi['label'] == 'Basophil'].count()[1]
Neutrophil = train_multi[train_multi['label'] == 'Neutrophil'].count()[1]
Lymphocyte = train_multi[train_multi['label'] == 'Lymphocyte'].count()[1]
total = len(train_multi)

print('Count of sample in each class is: \n')
print(f'Monocyte: {Monocyte} ({round(Monocyte/total,2)*100}%)')
print(f'Eosinophil: {Eosinophil} ({round(Eosinophil/total,2)*100}%)')
print(f'Basophil: {Basophil} ({round(Basophil/total,2)*100}%) ')
print(f'Neutrophil: {Neutrophil} ({round(Neutrophil/total,2)*100}%)')
print(f'Lymphocyte: {Lymphocyte} ({round(Lymphocyte/total,2)*100}%)')

In [ ]:
listFolders = os.listdir(image_dir)
pathList = []
for i in listFolders:
  pathList.append(os.path.join(image_dir,i))

Monocyte = pathList[0]
Eosinophil = pathList[1]
Basophil = pathList[2]
Neutrophil = pathList[3]
Lymphocyte = pathList[4]

In [ ]:
trainImage, trainLabel = train_multi['imagePath'], train_multi['label']
train = pd.DataFrame({'train_imagePath': trainImage, 'train_label': trainLabel})
train

## Test Train Spling

In [ ]:
#Training and Validation Split
df_train, df_val = train_test_split(train, test_size=0.25, random_state=42, shuffle=True, stratify=train['train_label'])
df_train

In [ ]:
#Training and Validation Split
df_test, df_val = train_test_split(df_val, test_size=0.4, random_state=42, shuffle=True, stratify=df_val['train_label'])
df_val

In [ ]:
df_test

## Data Generator Function

In [ ]:
# Define image size and parameters
batch_size = 64
img_height = 128
img_width =  128

image_size = (img_height, img_width)
input_shape = (img_height, img_width, 3)

In [ ]:
# Create an instance of ImageDataGenerator for Traing set only
train_datagen = ImageDataGenerator(
        rescale=1./255)#,brightness_range=[1,1],featurewise_std_normalization=True,featurewise_center=True,horizontal_flip=True,
    #vertical_flip=True)
#,height_shift_range=0.0, rotation_range=60, fill_mode='wrap')

In [ ]:
ds_trn = train_datagen.flow_from_dataframe(df_train,
  x_col='train_imagePath',
  y_col='train_label',
  target_size=input_shape[:2], # Resize images to the specified size
  class_mode='categorical', # For multi-class classification
  shuffle = False,
  batch_size=batch_size,
  keep_aspect_ratio=True)

In [ ]:
# Create an instance of ImageDataGenerator for Validation and Test sets
test_datagen = ImageDataGenerator(
        rescale=1./255)
#,brightness_range=[1,1],featurewise_std_normalization=True,featurewise_center=True,horizontal_flip=True,
    #vertical_flip=True)
  #height_shift_range=0.0, rotation_range=60, fill_mode='wrap')

In [ ]:
ds_val = test_datagen.flow_from_dataframe(df_val,
 x_col='train_imagePath',
  y_col='train_label',
  target_size=input_shape[:2], # Resize images to the specified size
  class_mode='categorical', # For multi-class classification
  shuffle = False,
  color_mode='rgb',
  batch_size=batch_size,
  keep_aspect_ratio=True)

In [ ]:
ds_test = test_datagen.flow_from_dataframe(df_test,
 x_col='train_imagePath',
  y_col='train_label',
  target_size=input_shape[:2], # Resize images to the specified size
  class_mode='categorical', # For multi-class classification
  shuffle = False,
  color_mode='rgb',
  batch_size=batch_size,
  keep_aspect_ratio=True)

In [ ]:
class_names = ds_trn.class_indices.keys()
class_names

#Show Sample Image

In [ ]:
def showSample(data, batchsize=25):
  classes = list(data.class_indices.keys()) # get the classes of the data
  image, labels = next(data)

  length = len(labels)
  sample = min(length,batchsize)

  plt.figure(figsize=(round(batchsize*2), round(batchsize*2)))
  m = 0
  b = batchsize
  while b>1:
    m+=1
    b = b/2

  for i in range(sample):
    plt.subplot(m, m, i+1)
    plt.imshow(image[i])
    index = np.argmax(labels[i])
    plt.title(classes[index],fontsize = m*6)
    plt.axis('off')

  plt.show()

In [ ]:
showSample(ds_trn)

In [ ]:
showSample(ds_test)

# Early Stopping

In [ ]:
# Define CallBack
class FinalCallBack(tf.keras.callbacks.Callback):
 def on_epoch_end(self, epoch, logs=None):
  print("\n****************\n\nThe average loss for epoch {} is{:7.3f} and accuracy is{:7.3f}.".format(epoch+1, logs["loss"], logs["accuracy"]),
  "\n\n***************\n")

EarlyStopping = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy',
  mode='max',
  min_delta = 0.01,
  baseline = 0.5,
  patience = 10,
  restore_best_weights = True)

#Model Fit

## Basic CNN Model

In [ ]:
# Create the Sequential model with modified architecture
model = tf.keras.Sequential([
  #for equal regularization
  tf.keras.layers.SeparableConv2D(filters=64, kernel_size=(3, 3), strides=(1, 1), activation="tanh", input_shape=input_shape),
  tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
  tf.keras.layers.Conv2D(filters=64, kernel_size=(3, 3), strides=(1, 1), activation="relu"),
  tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
  tf.keras.layers.SeparableConv2D(filters=128, kernel_size=(3,3), strides = (1,1), activation="relu",padding='same'),
  tf.keras.layers.MaxPooling2D(pool_size=(2, 2),padding='same'),
  tf.keras.layers.SpatialDropout2D(rate=0.1),
  tf.keras.layers.Conv2D(filters=128, kernel_size=(3,3), strides = (1,1), activation="relu",padding='same'),
  tf.keras.layers.MaxPooling2D(pool_size=(2, 2),padding='same'),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128,activation='elu'),
  tf.keras.layers.Dropout(rate=0.2),
  tf.keras.layers.Dense(64,activation='relu'),
  tf.keras.layers.Dense(5, activation='softmax')
], name='CNNModel')
# Compile the model with the modified optimizer
#sgd_optimizer = tf.keras.optimizers.SGD(learning_rate=0.01)
model.compile(optimizer='nadam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

In [ ]:
keras.utils.plot_model(model, show_shapes=True)

In [ ]:
fit1 = model.fit(ds_trn,
  epochs= 50,
  batch_size = 64,
  verbose= 1,
  callbacks=[FinalCallBack(),EarlyStopping], validation_data = ds_val)

In [ ]:
def plot_train_result(fit):
  a = fit.history['accuracy']
  b = fit.history['val_accuracy']
  c = fit.history['loss']
  d = fit.history['val_loss']

  best_acc = np.argmax(b)
  high_acc = b[best_acc]

  best_loss = np.argmin(d)
  low_loss = d[best_loss]

  plt.figure(figsize=(10,5))
  plt.subplot(1,2,1)
  plt.plot(a, label='Tarining Accuracy')
  plt.plot(b, label='Validation Accuracy')
  plt.scatter(best_acc+1, high_acc, s= 150, c= 'red', label= f'Best Epoch = {best_acc+1}')
  plt.xlabel('Epoch')
  plt.ylabel('Accuracy')
  plt.title('Tarining and Validation Accuracy')
  plt.legend()
  plt.grid()


  plt.subplot(1,2,2)
  plt.plot(c, label='Traing Loss')
  plt.plot(d, label='validation Loss')
  plt.scatter(best_loss+1, low_loss, s= 150, c= 'red', label= f'Best Epoch = {best_loss+1}')
  plt.xlabel('Epoch')
  plt.ylabel('Loss')
  plt.title('Tarining and Validation Loss')
  plt.grid()
  plt.legend()

In [ ]:
plot_train_result(fit1)

In [ ]:
# EVALUATE TEST SET PERFORMANCE
# YOUR CODE HERE (THIS SHOULD BE A SINGLE LINE OF CODE, RESULTING IN PRINTOUT OF TEST-SET PERFORMANCE VALUE)
model.evaluate(ds_test)

In [ ]:
y_predict = model.predict(ds_test)
y_predict = np.argmax(y_predict, axis=1)
class_names = ds_trn.class_indices.keys()
print(classification_report(ds_test.classes, y_predict, target_names=class_names))

In [ ]:
skplt.metrics.plot_confusion_matrix(ds_test.classes, y_predict,
                                      normalize = False,
                                      title = "Confusion Matrix",
                                    figsize = (10, 10))

In [ ]:
skplt.metrics.plot_confusion_matrix(ds_test.classes, y_predict,
                                      normalize = True,
                                      title = "Normalized Confusion Matrix",
                                    figsize = (10, 10))

## Base CNN Model with more parameters

In [ ]:
CNNA1 = tf.keras.Sequential([
 tf.keras.layers.SeparableConv2D(filters=64, kernel_size=(3, 3), strides=(1, 1), activation="tanh", input_shape=input_shape),
 tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
 tf.keras.layers.Conv2D(filters=128, kernel_size=(3,3), strides = (1,1), activation="relu",padding='same'),
 tf.keras.layers.MaxPooling2D(pool_size=(2, 2),padding='same'),
 tf.keras.layers.SpatialDropout2D(rate=0.1),
 tf.keras.layers.Conv2D(filters=128, kernel_size=(3,3), strides = (1,1), activation="relu",padding='same'),
 tf.keras.layers.MaxPooling2D(pool_size=(2, 2),padding='same'),
 tf.keras.layers.Flatten(),
 tf.keras.layers.Dense(128,activation='elu'),
 tf.keras.layers.Dropout(rate=0.2),
 tf.keras.layers.Dense(32,activation='relu'),
 tf.keras.layers.Dense(5, activation='softmax')
], name='CNNModel')


CNNA1.compile(optimizer='nadam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
CNNA1.summary()

In [ ]:
keras.utils.plot_model(CNNA1, show_shapes=True)

In [ ]:
fit2 = CNNA1.fit(ds_trn,
  epochs= 50,
  batch_size = 64,
  verbose= 1,
  callbacks=[FinalCallBack(),EarlyStopping],
  validation_data= ds_val)

In [ ]:
plot_train_result(fit2)

In [ ]:
# EVALUATE TEST SET PERFORMANCE
# YOUR CODE HERE (THIS SHOULD BE A SINGLE LINE OF CODE, RESULTING IN PRINTOUT OF TEST-SET PERFORMANCE VALUE)
CNNA1.evaluate(ds_test)

In [ ]:
y_predict = CNNA1.predict(ds_test)
y_predict = np.argmax(y_predict, axis=1)
class_names = ds_trn.class_indices.keys()
print(classification_report(ds_test.classes, y_predict, target_names=class_names))

In [ ]:
skplt.metrics.plot_confusion_matrix(ds_test.classes, y_predict,
                                      normalize = False,
                                      title = "Confusion Matrix",
                                    figsize = (10, 10))


In [ ]:
skplt.metrics.plot_confusion_matrix(ds_test.classes, y_predict,
                                      normalize = True,
                                      title = "Nomalized Confusion Matrix",
                                    figsize = (10, 10))

## Base CNN Model with Batch Normalization

In [ ]:
CNNH2N = tf.keras.Sequential([
 tf.keras.layers.SeparableConv2D(filters=32, kernel_size=(3, 3), strides=(1, 1), activation="tanh", input_shape=input_shape),
 tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
 BatchNormalization(),
 tf.keras.layers.Conv2D(filters=64, kernel_size=(3, 3), strides=(1, 1), activation="relu"),
 tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
 BatchNormalization(),
 tf.keras.layers.SeparableConv2D(filters=128, kernel_size=(3,3), strides = (1,1), activation="relu",padding='same'),
 tf.keras.layers.MaxPooling2D(pool_size=(2, 2),padding='same'),
 BatchNormalization(),
 tf.keras.layers.SpatialDropout2D(rate=0.1),
 tf.keras.layers.Conv2D(filters=128, kernel_size=(3,3), strides = (1,1), activation="relu",padding='same'),
 tf.keras.layers.MaxPooling2D(pool_size=(2, 2),padding='same'),
 BatchNormalization(),
 tf.keras.layers.Flatten(),
 tf.keras.layers.Dense(128,activation='elu'),
 BatchNormalization(),
 tf.keras.layers.Dropout(rate=0.2),
 tf.keras.layers.Dense(32,activation='relu'),
 BatchNormalization(),
 tf.keras.layers.Dense(5, activation='softmax')
], name='CNNModel')


CNNH2N.compile(optimizer='nadam', loss='categorical_crossentropy', metrics=['accuracy'])
CNNH2N.summary()

CNNH2N.summary()

In [ ]:
keras.utils.plot_model(CNNH2N, show_shapes=True)

In [ ]:
fit3 = CNNH2N.fit(ds_trn,
  epochs= 50,
  batch_size = 64,
  verbose= 1,
  callbacks=[FinalCallBack(),EarlyStopping],
  validation_data= ds_val)

In [ ]:
# EVALUATE TEST SET PERFORMANCE
# YOUR CODE HERE (THIS SHOULD BE A SINGLE LINE OF CODE, RESULTING IN PRINTOUT OF TEST-SET PERFORMANCE VALUE)
CNNH2N.evaluate(ds_test)

In [ ]:
y_predict = CNNH2N.predict(ds_test)
y_predict = np.argmax(y_predict, axis=1)
class_names = ds_trn.class_indices.keys()
print(classification_report(ds_test.classes, y_predict, target_names=class_names))

In [ ]:
skplt.metrics.plot_confusion_matrix(ds_test.classes, y_predict,
                                      normalize = False,
                                      title = "Confusion Matrix",
                                    figsize = (10, 10))

In [ ]:
skplt.metrics.plot_confusion_matrix(ds_test.classes, y_predict,
                                      normalize = True,
                                      title = "Nomalized Confusion Matrix",
                                    figsize = (10, 10))

## VGG-Model

In [ ]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras import models,optimizers, regularizers
# Define the batch size
batch_size = 64
# Define the number of classes in your dataset
num_classes = 5
# Create the VGGNet model with BatchNormalization and Dropout
def create_vgg_model(input_shape, num_classes):
 vgg_model = VGG16(weights='imagenet', include_top=False, input_shape=input_shape)
 # Freeze the VGG16 layers
 for layer in vgg_model.layers:
  layer.trainable = False
 model = models.Sequential()
 model.add(vgg_model)
 model.add(layers.Flatten())
 model.add(layers.Dense(4096, activation='relu'))
 model.add(layers.BatchNormalization())
 model.add(layers.Dropout(0.5))

 model.add(layers.Dense(4096, activation='relu'))
 model.add(layers.BatchNormalization())
 model.add(layers.Dropout(0.5))

 model.add(layers.Dense(num_classes, activation='softmax'))
 return model
# Create the VGGNet model
modelvgg = create_vgg_model(input_shape, num_classes)

In [ ]:
# Compile the model
#optimizer = optimizers.Adam(learning_rate=0.001)
modelvgg.compile(optimizer='nadam', loss='categorical_crossentropy', metrics=['accuracy'])
# Print the model summary
modelvgg.summary()

In [ ]:
keras.utils.plot_model(modelvgg, show_shapes=True)

In [ ]:
fit4 = modelvgg.fit(ds_trn,
  epochs= 50,
  batch_size = 64,
  verbose= 1,
  callbacks=[FinalCallBack(),EarlyStopping],
  validation_data= ds_val)

In [ ]:
plot_train_result(fit4)

In [ ]:
# EVALUATE TEST SET PERFORMANCE
# YOUR CODE HERE (THIS SHOULD BE A SINGLE LINE OF CODE, RESULTING IN PRINTOUT OF TEST-SET PERFORMANCE VALUE)
modelvgg.evaluate(ds_test)

In [ ]:
y_predict = modelvgg.predict(ds_test)
y_predict = np.argmax(y_predict, axis=1)
class_names = ds_trn.class_indices.keys()
print(classification_report(ds_test.classes, y_predict, target_names=class_names))

In [ ]:
skplt.metrics.plot_confusion_matrix(ds_test.classes, y_predict,
                                      normalize = False,
                                      title = "Confusion Matrix",
                                    figsize = (10, 10))

In [ ]:
skplt.metrics.plot_confusion_matrix(ds_test.classes, y_predict,
                                      normalize = True,
                                      title = "Normalized Confusion Matrix",
                                    figsize = (10, 10))

## Resnet Model

In [ ]:
def identity_block(input, filters):
 filter1, filter2, filter3 = filters
 x = tf.keras.layers.Conv2D(filter1, (1,1))(input)
 x = tf.keras.layers.BatchNormalization()(x)
 x = tf.keras.activations.relu(x)
 x = tf.keras.layers.Conv2D(filter2, (3,3), padding='same')(x)
 x = tf.keras.layers.BatchNormalization()(x)
 x = tf.keras.activations.relu(x)
 x = tf.keras.layers.Conv2D(filter3, (1,1))(x)
 x = tf.keras.layers.BatchNormalization()(x)
 x = tf.keras.layers.add([x, input])
 x = tf.keras.activations.relu(x)
 return x

In [ ]:
def conv_block(input, filters, strides):
 filter1, filter2, filter3 = filters
 x = tf.keras.layers.Conv2D(filter1, (1,1), strides=strides)(input)
 x = tf.keras.layers.BatchNormalization()(x)
 x = tf.keras.activations.relu(x)
 x = tf.keras.layers.Conv2D(filter2, (3,3), padding='same')(x)
 x = tf.keras.layers.BatchNormalization()(x)
 x = tf.keras.activations.relu(x)
 x = tf.keras.layers.Conv2D(filter3, (1,1))(x)
 x = tf.keras.layers.BatchNormalization()(x)
 tmp = tf.keras.layers.Conv2D(filter3, (1,1), strides=strides)(input)
 tmp = tf.keras.layers.BatchNormalization()(tmp)
 x = layers.add([x, tmp])
 x = tf.keras.activations.relu(x)
 return x

In [ ]:
model_res50 = tf.keras.Sequential()

In [ ]:
def resnet50():
 input = tf.keras.Input(shape = input_shape)
 x = tf.keras.layers.ZeroPadding2D((3,3))(input)
 x = tf.keras.layers.Conv2D(64, (7,7), strides=(2,2))(input)
 x = tf.keras.layers.BatchNormalization()(x)
 x = tf.keras.activations.relu(x)
 x = tf.keras.layers.MaxPooling2D(pool_size=(3,3), strides=(2,2))(x)
 x = conv_block(x, [64,64,256], strides=(1,1))
 x = identity_block(x, [64,64,256])
 x = identity_block(x, [64,64,256])
 x = conv_block(x, [128,128,512], strides=(2,2))
 x = identity_block(x, [128,128,512])
 x = identity_block(x, [128,128,512])
 x = identity_block(x, [128,128,512])
 x = conv_block(x, [256,256,1024], strides=(2,2))
 x = identity_block(x, [256,256,1024])
 x = identity_block(x, [256,256,1024])
 x = identity_block(x, [256,256,1024])
 x = identity_block(x, [256,256,1024])
 x = identity_block(x, [256,256,1024])
 x = conv_block(x, [512,512,2048], strides=(2,2))
 x = identity_block(x, [512,512,2048])
 x = identity_block(x, [512,512,2048])
 x = tf.keras.layers.AveragePooling2D(pool_size=(3,3),padding='same')(x)
 x = tf.keras.layers.Flatten()(x)
 output = tf.keras.layers.Dense(5, activation='softmax')(x)
 model = tf.keras.models.Model(input, output)
 return model


In [ ]:
resNet50 = resnet50()
resNet50.compile(optimizer='nadam', loss='categorical_crossentropy', metrics=['accuracy'])
resNet50.summary()

In [ ]:
keras.utils.plot_model(resNet50, show_shapes=True)

In [ ]:
a = resNet50.fit(ds_trn,
  epochs= 50,
  batch_size = 64,
  verbose= 1,
  callbacks=[FinalCallBack(),EarlyStopping],
  validation_data= ds_val)

In [ ]:
# EVALUATE TEST SET PERFORMANCE
# YOUR CODE HERE (THIS SHOULD BE A SINGLE LINE OF CODE, RESULTING IN PRINTOUT OF TEST-SET PERFORMANCE VALUE)
resNet50.evaluate(ds_test)

In [ ]:
y_predict = modelvgg.predict(ds_test)
y_predict = np.argmax(y_predict, axis=1)
class_names = ds_trn.class_indices.keys()
print(classification_report(ds_test.classes, y_predict, target_names=class_names))

In [ ]:
skplt.metrics.plot_confusion_matrix(ds_test.classes, y_predict,
                                      normalize = False,
                                      title = "Confusion Matrix",
                                    figsize = (10, 10))

In [ ]:
skplt.metrics.plot_confusion_matrix(ds_test.classes, y_predict,
                                      normalize = True,
                                      title = "Nomalized Confusion Matrix",
                                    figsize = (10, 10))